In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  4
/device:GPU:0


2022-07-17 08:44:12.855096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-17 08:44:14.259022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 38417 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2022-07-17 08:44:14.260224: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:1 with 38417 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2022-07-17 08:44:14.260860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:2 with 38417 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:8

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import h5py

import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Input, Lambda, LeakyReLU, Add, Dense, Activation, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform, constant, TruncatedNormal

%matplotlib inline

In [4]:
with h5py.File('../processed_data/np_gan_standard.h5', 'r') as hf:
    data_lr = hf['np_lr'][:]
    data_lr_mean = hf['np_lr_mean'][:]
    data_lr_stddev = hf['np_lr_stddev'][:]
    data_hr = hf['np_hr'][:]
    data_hr_mean = hf['np_hr_mean'][:]
    data_hr_stddev = hf['np_hr_stddev'][:]

In [5]:
print(data_lr.shape)
print(data_lr_mean)
print(data_lr_stddev)
print('\n')
print(data_hr.shape)
print(data_hr_mean)
print(data_hr_stddev)  

(8520, 96, 96, 2)
[ 0.7051484 -1.0147774]
[3.1869051 2.8827915]


(8520, 192, 192, 2)
[ 0.701198  -1.0068085]
[3.149407  2.8781955]


In [6]:
#First split data into train+validation and test set
x_train, x_test, y_train, y_test = train_test_split(data_lr, data_hr, test_size=0.2, random_state=42)

#Next split training again into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

print(np.max(x_train), np.max(x_val), np.max(x_test), np.min(x_train), np.min(x_val), np.min(x_test))
print(np.max(y_train), np.max(y_val), np.max(y_test), np.min(y_train), np.min(y_val), np.min(y_test))

(5112, 96, 96, 2)
(1704, 96, 96, 2)
(1704, 96, 96, 2)
(5112, 192, 192, 2)
(1704, 192, 192, 2)
(1704, 192, 192, 2)
6.890482 5.989441 5.7024393 -5.4777956 -5.0524907 -5.5057893
7.035497 5.8600492 6.0442076 -5.315754 -5.2317853 -5.2996855


In [7]:
def generator(input_shape = (96, 96, 2), nf = 64, r = 2):
    """
    Arguments:
    input_shape -- shape of the images of the dataset, H*W*C
    nf -- integer, the number of filters in all convT layer before super-resolution step
    r -- integer, resolution ratio between output and input

    Returns:
    model -- a Model() instance in Keras
    """
    
    C0 = input_shape[2]
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Define kernel size and stride used
    k, stride = 3, 1
    
    # Shall we use a mirror padding and finally cutoff the edge, like the paper does? FIXME
    X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X_input)
    # Shall we use relu, or leaky_relu? FIXME
    X = Activation('relu')(X)

    skip_connection = X
    
    for i in range(16):
        X_shortcut = X
        
        X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
        X = Activation('relu')(X)
        X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
        X = Add()([X, X_shortcut])
        # Are we missing a relu activation here, if we follow the resnet paper? FIXME
    
    X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    X = Add()([X, skip_connection])
    
    # Start to perform sr
    nf_sr = (r**2) * nf
    X = Conv2DTranspose(filters=nf_sr, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    
    sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,r))
    X = sub_layer(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters=C0, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [8]:
gen_model = generator(input_shape = (96, 96, 2))
print(gen_model.summary())

2022-07-17 08:44:26.224909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38417 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2022-07-17 08:44:26.225286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38417 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2022-07-17 08:44:26.225652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38417 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2022-07-17 08:44:26.226034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 38417 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-40GB, pci bu

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 96, 96, 64)  1216        ['input_1[0][0]']                
 ose)                                                                                             
                                                                                                  
 activation (Activation)        (None, 96, 96, 64)   0           ['conv2d_transpose[0][0]']       
                                                                                                  
 conv2d_transpose_1 (Conv2DTran  (None, 96, 96, 64)  36928       ['activation[0][0]']         

In [9]:
def discriminator(input_shape = (192, 192, 2)):
    """
    Arguments:
    input_shape -- shape of the images of the dataset, H*W*C

    Returns:
    model -- a Model() instance in Keras
    """
    
    C0 = input_shape[2]
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    #conv1
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(X_input)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv2
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv3
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv4
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv5
    X = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv6
    X = Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv7
    X = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #conv8
    X = Conv2D(filters=256, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Flatten()(X)
    
    #first fully-connect
    k_init = TruncatedNormal(stddev=0.02)
    X = Dense(units=1024, kernel_initializer=k_init)(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    #second fully-connect, no activation FIXME
    X = Dense(units=1, kernel_initializer=k_init)(X)
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [10]:
disc_model = discriminator(input_shape = (192, 192, 2))
print(disc_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 192, 192, 2)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 32)      608       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 192, 192, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 96, 96, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 96, 64)        18496     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 96, 96, 64)        0   

In [11]:
def generator_only_loss(x_HR, x_SR):
    content_loss = tf.reduce_mean((x_HR - x_SR)**2)   
    return content_loss

def generator_loss(x_HR, x_SR, d_SR, alpha_advers=0.001):
    
    content_loss = tf.reduce_mean((x_HR - x_SR)**2)
    g_advers_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_SR, labels=tf.ones_like(d_SR)))
    g_loss = content_loss + alpha_advers * g_advers_loss
    
    return g_loss, content_loss, g_advers_loss

#def discriminator_loss(d_HR, d_SR):
#   
#    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.concat([d_HR, d_SR], axis=0),
#                                                                  labels=tf.concat([tf.ones_like(d_HR), tf.zeros_like(d_SR)], axis=0)))

def discriminator_loss(d_HR, d_SR):
    
    advers_perf = [tf.reduce_mean(tf.cast(tf.sigmoid(d_HR) > 0.5, tf.float32)), # % true positive
                       tf.reduce_mean(tf.cast(tf.sigmoid(d_SR) < 0.5, tf.float32)), # % true negative
                       tf.reduce_mean(tf.cast(tf.sigmoid(d_SR) > 0.5, tf.float32)), # % false positive
                       tf.reduce_mean(tf.cast(tf.sigmoid(d_HR) < 0.5, tf.float32))] # % false negative
    
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.concat([d_HR, d_SR], axis=0),
                                                                  labels=tf.concat([tf.ones_like(d_HR), tf.zeros_like(d_SR)], axis=0))), advers_perf

In [12]:
gen_model.load_weights("generator_pretrain/saved-model-0720.hdf5")

In [13]:
print(generator_only_loss(y_test, gen_model.predict(x_test)))

2022-07-17 08:44:36.185950: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8302


54/54 [==============================] - 7s 31ms/step
tf.Tensor(0.011877065, shape=(), dtype=float32)


In [ ]:
from time import time

@tf.function
def train_step(generator, discriminator, generator_optimizer, discriminator_optimizer, batch_LR, batch_HR, alpha_advers=0.001):
    
    g_count = 0
    d_loss = tf.constant(0.0)
    d_HR = discriminator(batch_HR, training=False)
    while(d_loss < tf.constant(0.45) and g_count < 40):
        g_count += 1
        with tf.GradientTape() as gen_tape:
            batch_SR = generator(batch_LR, training=True)
            d_SR = discriminator(batch_SR, training=False)
            g_loss, content_loss, g_advers_loss = generator_loss(batch_HR, batch_SR, d_SR, alpha_advers=alpha_advers)

        grad_of_gen = gen_tape.gradient(g_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(grad_of_gen, generator.trainable_variables))
        d_loss, _ = discriminator_loss(d_HR, d_SR)

    d_count = 0
    d_loss = tf.constant(100.0)
    while(d_loss > tf.constant(0.65) and d_count < 40):
        d_count += 1
        with tf.GradientTape() as disc_tape:
            batch_SR = generator(batch_LR, training=False)
            d_HR = discriminator(batch_HR, training=True)
            d_SR = discriminator(batch_SR, training=True)
            d_loss, _ = discriminator_loss(d_HR, d_SR)

        grad_of_disc = disc_tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(grad_of_disc, discriminator.trainable_variables))
       
    return g_loss, d_loss, g_count, d_count

def train(gen_model, disc_model, epochs=20, batch_size=128, alpha_advers=0.001):
    '''
        This method trains the generator and disctiminator adversarially
        Notice the two model should be argument of this function
        output: generator model and discriminator model
    '''

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    batch_count = tf.data.experimental.cardinality(train_dataset)
    
    g_opt = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_opt = tf.keras.optimizers.Adam(learning_rate=4e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Start training
    print('Training network ...')
    for epoch in range(1, epochs+1):
        print('Epoch: %d' %(epoch))
        start_time = time()
        epoch_g_loss, epoch_d_loss, N, g_count_tot, d_count_tot = 0, 0, 0, 0, 0
        
        for batch_idx, (batch_LR, batch_HR) in enumerate(train_dataset):
            N_batch = batch_LR.shape[0]
            g_loss, d_loss, g_count, d_count = train_step(gen_model, disc_model, g_opt, d_opt, batch_LR, batch_HR, alpha_advers)
            
            epoch_g_loss += g_loss * N_batch
            epoch_d_loss += d_loss * N_batch
            N += N_batch
            g_count_tot += g_count
            d_count_tot += d_count

        epoch_g_loss = epoch_g_loss / N       
        epoch_d_loss = epoch_d_loss / N       
        
        val_SR = gen_model.predict(x_val, verbose=0)
        val_d_HR = disc_model.predict(y_val, verbose=0)
        val_d_SR = disc_model.predict(val_SR, verbose=0)
        
        val_g_loss, val_content_loss, val_advers_loss = generator_loss(y_val, val_SR, val_d_SR, alpha_advers)
        val_d_loss, val_disc_perf = discriminator_loss(val_d_HR, val_d_SR)
        
        print('Epoch generator loss = %.6f, discriminator loss = %.6f, g_count = %d, d_count = %d' %(epoch_g_loss, epoch_d_loss, g_count_tot, d_count_tot))
        print('Epoch val: g_loss = %.6f, d_loss = %.6f, content_loss = %.6f, advers_loss = %.6f' \
              %(val_g_loss, val_d_loss, val_content_loss, val_advers_loss))
        print('Epoch val disc perf: TP=%.5f, TN=%.5f, FP=%.5f, FN=%.5f' %(val_disc_perf[0], val_disc_perf[1], val_disc_perf[2], val_disc_perf[3]))
        print('Epoch took %.2f seconds\n' %(time() - start_time), flush=True)
        
        if (epoch % 10 == 0):
            gen_model.save('gan_train/gen-model-epoch-%d.h5' %(epoch))
            disc_model.save('gan_train/disc-model-epoch-%d.h5' %(epoch))

    print('Done.')

    return gen_model, disc_model

In [17]:
gen_model_final, disc_model_final = train(gen_model, disc_model, epochs=100, batch_size=128, alpha_advers=0.001)

Training network ...
Epoch: 1
Epoch generator loss = 0.014066, discriminator loss = 0.633818, g_count = 40, d_count = 50
Epoch val: g_loss = 0.013259, d_loss = 0.643441, content_loss = 0.012493, advers_loss = 0.765912
Epoch val disc perf: TP=0.63850, TN=0.73357, FP=0.26643, FN=0.36150
Epoch took 37.96 seconds

Epoch: 2
Epoch generator loss = 0.011368, discriminator loss = 0.637378, g_count = 40, d_count = 40
Epoch val: g_loss = 0.012871, d_loss = 0.627145, content_loss = 0.012039, advers_loss = 0.831699
Epoch val disc perf: TP=0.58392, TN=0.86854, FP=0.13146, FN=0.41608
Epoch took 27.92 seconds

Epoch: 3
Epoch generator loss = 0.011144, discriminator loss = 0.617676, g_count = 40, d_count = 40
Epoch val: g_loss = 0.012879, d_loss = 0.596519, content_loss = 0.012044, advers_loss = 0.834565
Epoch val disc perf: TP=0.74413, TN=0.79519, FP=0.20481, FN=0.25587
Epoch took 27.98 seconds

Epoch: 4
Epoch generator loss = 0.011159, discriminator loss = 0.593160, g_count = 40, d_count = 40
Epoch 

In [14]:
from time import time

@tf.function
def train_step(generator, discriminator, generator_optimizer, discriminator_optimizer, batch_LR, batch_HR, alpha_advers=0.001):
    
    g_count = 0
    d_loss = tf.constant(0.0)
    d_HR = discriminator(batch_HR, training=False)
    while(d_loss < tf.constant(0.5) and g_count < 40):
        g_count += 1
        with tf.GradientTape() as gen_tape:
            batch_SR = generator(batch_LR, training=True)
            d_SR = discriminator(batch_SR, training=False)
            g_loss, content_loss, g_advers_loss = generator_loss(batch_HR, batch_SR, d_SR, alpha_advers=alpha_advers)

        grad_of_gen = gen_tape.gradient(g_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(grad_of_gen, generator.trainable_variables))
        d_loss, _ = discriminator_loss(d_HR, d_SR)

    d_count = 0
    d_loss = tf.constant(100.0)
    while(d_loss > tf.constant(0.7) and d_count < 40):
        d_count += 1
        with tf.GradientTape() as disc_tape:
            batch_SR = generator(batch_LR, training=False)
            d_HR = discriminator(batch_HR, training=True)
            d_SR = discriminator(batch_SR, training=True)
            d_loss, _ = discriminator_loss(d_HR, d_SR)

        grad_of_disc = disc_tape.gradient(d_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(grad_of_disc, discriminator.trainable_variables))
       
    return g_loss, d_loss, g_count, d_count

def train(gen_model, disc_model, epochs=20, batch_size=128, alpha_advers=0.001):
    '''
        This method trains the generator and disctiminator adversarially
        Notice the two model should be argument of this function
        output: generator model and discriminator model
    '''

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    batch_count = tf.data.experimental.cardinality(train_dataset)
    
    g_opt = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_opt = tf.keras.optimizers.Adam(learning_rate=4e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Start training
    print('Training network ...')
    for epoch in range(1, epochs+1):
        print('Epoch: %d' %(epoch))
        start_time = time()
        epoch_g_loss, epoch_d_loss, N, g_count_tot, d_count_tot = 0, 0, 0, 0, 0
        
        for batch_idx, (batch_LR, batch_HR) in enumerate(train_dataset):
            N_batch = batch_LR.shape[0]
            g_loss, d_loss, g_count, d_count = train_step(gen_model, disc_model, g_opt, d_opt, batch_LR, batch_HR, alpha_advers)
            
            epoch_g_loss += g_loss * N_batch
            epoch_d_loss += d_loss * N_batch
            N += N_batch
            g_count_tot += g_count
            d_count_tot += d_count

        epoch_g_loss = epoch_g_loss / N       
        epoch_d_loss = epoch_d_loss / N       
        
        val_SR = gen_model.predict(x_val, verbose=0)
        val_d_HR = disc_model.predict(y_val, verbose=0)
        val_d_SR = disc_model.predict(val_SR, verbose=0)
        
        val_g_loss, val_content_loss, val_advers_loss = generator_loss(y_val, val_SR, val_d_SR, alpha_advers)
        val_d_loss, val_disc_perf = discriminator_loss(val_d_HR, val_d_SR)
        
        print('Epoch generator loss = %.6f, discriminator loss = %.6f, g_count = %d, d_count = %d' %(epoch_g_loss, epoch_d_loss, g_count_tot, d_count_tot))
        print('Epoch val: g_loss = %.6f, d_loss = %.6f, content_loss = %.6f, advers_loss = %.6f' \
              %(val_g_loss, val_d_loss, val_content_loss, val_advers_loss))
        print('Epoch val disc perf: TP=%.5f, TN=%.5f, FP=%.5f, FN=%.5f' %(val_disc_perf[0], val_disc_perf[1], val_disc_perf[2], val_disc_perf[3]))
        print('Epoch took %.2f seconds\n' %(time() - start_time), flush=True)
        
        if (epoch % 2 == 0):
            gen_model.save('gan_train/gen-model-epoch-%d.h5' %(epoch+280))
            disc_model.save('gan_train/disc-model-epoch-%d.h5' %(epoch+280))

    print('Done.')

    return gen_model, disc_model

In [15]:
gen_model = tf.keras.models.load_model('gan_train/gen-model-epoch-280.h5')
disc_model = tf.keras.models.load_model('gan_train/disc-model-epoch-280.h5')

val_SR = gen_model.predict(x_val, verbose=0)
val_d_HR = disc_model.predict(y_val, verbose=0)
val_d_SR = disc_model.predict(val_SR, verbose=0)

val_g_loss, val_content_loss, val_advers_loss = generator_loss(y_val, val_SR, val_d_SR, 0.001)
val_d_loss, val_disc_perf = discriminator_loss(val_d_HR, val_d_SR)

print('Epoch val: g_loss = %.6f, d_loss = %.6f, content_loss = %.6f, advers_loss = %.6f' \
      %(val_g_loss, val_d_loss, val_content_loss, val_advers_loss))
print('Epoch val disc perf: TP=%.5f, TN=%.5f, FP=%.5f, FN=%.5f' %(val_disc_perf[0], val_disc_perf[1], val_disc_perf[2], val_disc_perf[3]))

2022-07-17 08:45:54.882581: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch val: g_loss = 0.015782, d_loss = 0.500255, content_loss = 0.014153, advers_loss = 1.628517
Epoch val disc perf: TP=0.63380, TN=0.84624, FP=0.15376, FN=0.36620


In [16]:
gen_model_final, disc_model_final = train(gen_model, disc_model, epochs=40, batch_size=128, alpha_advers=0.001)

Training network ...
Epoch: 1
Epoch generator loss = 0.012713, discriminator loss = 0.498882, g_count = 386, d_count = 41
Epoch val: g_loss = 0.015188, d_loss = 0.468793, content_loss = 0.013782, advers_loss = 1.405567
Epoch val disc perf: TP=0.76995, TN=0.76819, FP=0.23181, FN=0.23005
Epoch took 149.65 seconds

Epoch: 2
Epoch generator loss = 0.011924, discriminator loss = 0.512821, g_count = 196, d_count = 40
Epoch val: g_loss = 0.015203, d_loss = 0.473740, content_loss = 0.013901, advers_loss = 1.302227
Epoch val disc perf: TP=0.80810, TN=0.73298, FP=0.26702, FN=0.19190
Epoch took 76.02 seconds

Epoch: 3
Epoch generator loss = 0.011821, discriminator loss = 0.513881, g_count = 180, d_count = 40
Epoch val: g_loss = 0.015326, d_loss = 0.501632, content_loss = 0.014083, advers_loss = 1.242618
Epoch val disc perf: TP=0.80751, TN=0.67019, FP=0.32981, FN=0.19249
Epoch took 70.80 seconds

Epoch: 4
Epoch generator loss = 0.011855, discriminator loss = 0.516255, g_count = 149, d_count = 40
E

In [18]:
for i in range(180, 220, 2):
    
    gen_model = tf.keras.models.load_model('gan_train/gen-model-epoch-{}.h5'.format(i+100))
    disc_model = tf.keras.models.load_model('gan_train/disc-model-epoch-{}.h5'.format(i))

    val_SR = gen_model.predict(x_val, verbose=0)
    val_d_HR = disc_model.predict(y_val, verbose=0)
    val_d_SR = disc_model.predict(val_SR, verbose=0)

    val_g_loss, val_content_loss, val_advers_loss = generator_loss(y_val, val_SR, val_d_SR, 0.001)
    val_d_loss, val_disc_perf = discriminator_loss(val_d_HR, val_d_SR)

    print('Model {}:'.format(i))
    print('Epoch val: g_loss = %.6f, d_loss = %.6f, content_loss = %.6f, advers_loss = %.6f' \
          %(val_g_loss, val_d_loss, val_content_loss, val_advers_loss))
    print('Epoch val disc perf: TP=%.5f, TN=%.5f, FP=%.5f, FN=%.5f' %(val_disc_perf[0], val_disc_perf[1], val_disc_perf[2], val_disc_perf[3]))
    
    test_SR = gen_model.predict(x_test, verbose=0)
    test_d_HR = disc_model.predict(y_test, verbose=0)
    test_d_SR = disc_model.predict(test_SR, verbose=0)

    test_g_loss, test_content_loss, test_advers_loss = generator_loss(y_test, test_SR, test_d_SR, 0.001)
    test_d_loss, test_disc_perf = discriminator_loss(test_d_HR, test_d_SR)

    print('Epoch test: g_loss = %.6f, d_loss = %.6f, content_loss = %.6f, advers_loss = %.6f' \
          %(test_g_loss, test_d_loss, test_content_loss, test_advers_loss))
    print('Epoch test disc perf: TP=%.5f, TN=%.5f, FP=%.5f, FN=%.5f' %(test_disc_perf[0], test_disc_perf[1], test_disc_perf[2], test_disc_perf[3]))

Model 180:
Epoch val: g_loss = 0.014371, d_loss = 1.374425, content_loss = 0.014153, advers_loss = 0.217657
Epoch val disc perf: TP=0.87383, TN=0.06631, FP=0.93369, FN=0.12617
Epoch test: g_loss = 0.014097, d_loss = 1.417458, content_loss = 0.013898, advers_loss = 0.198370
Epoch test disc perf: TP=0.86913, TN=0.05692, FP=0.94308, FN=0.13087
Model 182:
Epoch val: g_loss = 0.015203, d_loss = 0.473735, content_loss = 0.013901, advers_loss = 1.302238
Epoch val disc perf: TP=0.80810, TN=0.73239, FP=0.26761, FN=0.19190
Epoch test: g_loss = 0.014888, d_loss = 0.485003, content_loss = 0.013641, advers_loss = 1.247303
Epoch test disc perf: TP=0.80810, TN=0.72653, FP=0.27347, FN=0.19190
Model 184:
Epoch val: g_loss = 0.016013, d_loss = 0.629861, content_loss = 0.014322, advers_loss = 1.691400
Epoch val disc perf: TP=0.40082, TN=0.83862, FP=0.16138, FN=0.59918
Epoch test: g_loss = 0.015711, d_loss = 0.639831, content_loss = 0.014058, advers_loss = 1.652977
Epoch test disc perf: TP=0.38908, TN=0.8